In [1]:
import cv2
import os
import random
import numpy as np
from scipy.ndimage import gaussian_filter, map_coordinates
import torch
import torch.nn as nn
from sklearn.cluster import DBSCAN
from torch.utils.data import random_split, ConcatDataset
from torchvision.datasets import ImageFolder
from torchvision import transforms
from torch.utils.data import DataLoader
from sklearn.model_selection import train_test_split
from torch.utils.data.sampler import SubsetRandomSampler
from torch.optim import SGD
from torch.optim.lr_scheduler import ReduceLROnPlateau
from sklearn.metrics import precision_recall_fscore_support, accuracy_score
from tqdm import tqdm
import dlib
from PIL import Image
import wandb

In [2]:
class EmotionCNN(nn.Module):
    def __init__(self, num_classes=7):
        super(EmotionCNN, self).__init__()
        
        self.conv1 = nn.Conv2d(1, 64, kernel_size=5, stride=1, padding=0)
        self.relu1 = nn.ReLU()
        self.maxpool1 = nn.MaxPool2d(kernel_size=5, stride=2)
        
        self.conv2a = nn.Conv2d(64, 64, kernel_size=3, stride=1, padding=1)
        self.relu2a = nn.ReLU()
        self.conv2b = nn.Conv2d(64, 64, kernel_size=3, stride=1, padding=1)
        self.relu2b = nn.ReLU()
        self.avgpool2 = nn.AvgPool2d(kernel_size=3, stride=2)
        
        self.conv3a = nn.Conv2d(64, 128, kernel_size=3, stride=1, padding=1)
        self.relu3a = nn.ReLU()
        self.conv3b = nn.Conv2d(128, 128, kernel_size=3, stride=1, padding=1)
        self.relu3b = nn.ReLU()
        self.avgpool3 = nn.AvgPool2d(kernel_size=3, stride=2)
        
        # verify the output size of conv2 and conv3
        self.dummy_input = torch.randn(1, 1, 48, 48)
        self.dummy_output_size = self._get_conv_output_size(self.dummy_input)
        
        # update fc1 units based on feature map size
        self.fc1 = nn.Linear(self.dummy_output_size, 1024)
        self.relu_fc1 = nn.ReLU()
        self.dropout1 = nn.Dropout(0.2)
        
        self.fc2 = nn.Linear(1024, 1024)
        self.relu_fc2 = nn.ReLU()
        self.dropout2 = nn.Dropout(0.2)
        
        self.fc3 = nn.Linear(1024, num_classes)
        self.softmax = nn.Softmax(dim=1)

    def _get_conv_output_size(self, input_tensor):
        x = self.maxpool1(self.relu1(self.conv1(input_tensor)))
        x = self.relu2a(self.conv2a(x))
        x = self.relu2b(self.conv2b(x))
        x = self.avgpool2(x)
        x = self.relu3a(self.conv3a(x))
        x = self.relu3b(self.conv3b(x))
        x = self.avgpool3(x)
        return x.view(x.size(0), -1).size(1)

    def forward(self, x):
        x = self.maxpool1(self.relu1(self.conv1(x)))
        x = self.relu2a(self.conv2a(x))
        x = self.relu2b(self.conv2b(x))
        x = self.avgpool2(x)
        x = self.relu3a(self.conv3a(x))
        x = self.relu3b(self.conv3b(x))
        x = self.avgpool3(x)
        x = x.view(x.size(0), -1)
        x = self.dropout1(self.relu_fc1(self.fc1(x)))
        x = self.dropout2(self.relu_fc2(self.fc2(x)))
        x = self.softmax(self.fc3(x))
        return x

In [3]:
number_instances_over_under_sampling_ = 20000
batch_size_ = 48
epochs_ = 15

In [4]:
# transformation definition
transform = transforms.Compose([
    transforms.Grayscale(num_output_channels=1),
    transforms.Resize((48, 48)),
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])

dataset_root = r'C:\Users\marco\OneDrive\Documenti\CV_project\ComputerVisionProject\Data\emotions_aug_images'

# create an instance of ImageFolder with the transformations
dataset = ImageFolder(root=dataset_root, transform=transform)

# seed = 42
torch.manual_seed(42)

# extract the labels and the indices of the dataset
labels = [label for _, label in dataset.imgs]

# convert the list into a tensor
labels = torch.tensor(labels)

# calculate the number of instances for each class
counts = torch.bincount(labels)

# calculate the weights for each class
weights = 1.0 / counts.float()

# create a weight vector for each index in the dataset
sample_weights = weights[labels]

# set the number of samples for the train set and the test set
train_size = number_instances_over_under_sampling_ * 7 * 0.8
val_size = number_instances_over_under_sampling_ * 7 * 0.1
test_size = number_instances_over_under_sampling_ * 7 * 0.1

# crea un sampler per il train set and one for the test set
train_sampler = torch.utils.data.WeightedRandomSampler(sample_weights, int(train_size))
val_sampler = torch.utils.data.WeightedRandomSampler(sample_weights, int(val_size))
test_sampler = torch.utils.data.WeightedRandomSampler(sample_weights, int(test_size))

# create a dataloader for the train set and the test set with the corresponding samplers
train_loader = DataLoader(dataset, batch_size=batch_size_, sampler=train_sampler, num_workers=4)
val_loader = DataLoader(dataset, batch_size=batch_size_, sampler=val_sampler, num_workers=4)
test_loader = DataLoader(dataset, batch_size=batch_size_, sampler=test_sampler, num_workers=4)


In [5]:
# transformation definition
transform = transforms.Compose([
    transforms.Grayscale(num_output_channels=1),
    transforms.Resize((48, 48)),
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])

dataset_root = r'C:\Users\marco\OneDrive\Documenti\CV_project\ComputerVisionProject\Data\test_images_emotion'

# create an instance of ImageFolder with the transformations
dataset = ImageFolder(root=dataset_root, transform=transform)

# seed = 42
torch.manual_seed(42)

# extract the labels and the indices of the dataset
labels = [label for _, label in dataset.imgs]

# convert the list into a tensor
labels = torch.tensor(labels)

# calculate the number of instances for each class
counts = torch.bincount(labels)

# calculate the weights for each class
weights = 1.0 / counts.float()

# create a weight vector for each index in the dataset
sample_weights = weights[labels]

# set the number of samples for the train set and the test set
train_size = (number_instances_over_under_sampling_/10) * 7 * 0.1
val_size = (number_instances_over_under_sampling_/10) * 7 * 0.1
test_size = (number_instances_over_under_sampling_/10) * 7 * 0.8

# crea un sampler per il train set and one for the test set
train_sampler = torch.utils.data.WeightedRandomSampler(sample_weights, int(train_size))
val_sampler = torch.utils.data.WeightedRandomSampler(sample_weights, int(val_size))
test_sampler = torch.utils.data.WeightedRandomSampler(sample_weights, int(test_size))

# create a dataloader for the train set and the test set with the corresponding samplers
train_loader_ = DataLoader(dataset, batch_size=batch_size_, sampler=train_sampler, num_workers=4)
val_loader_ = DataLoader(dataset, batch_size=batch_size_, sampler=val_sampler, num_workers=4)
test_loader_ = DataLoader(dataset, batch_size=batch_size_, sampler=test_sampler, num_workers=4)

In [6]:
def calculate_metrics_per_class(true_labels, predicted_labels, label_mapping):
    unique_labels = list(label_mapping.keys())
    precision, recall, f1, support = precision_recall_fscore_support(true_labels, predicted_labels, labels=unique_labels)
    accuracy = accuracy_score(true_labels, predicted_labels)
    
    metrics_per_class = {}
    for i, idx in enumerate(unique_labels):
        metrics_per_class[idx] = {
            'precision': precision[i],
            'recall': recall[i],
            'f1': f1[i],
            'support': support[i]
        }

    return accuracy, metrics_per_class

# Funzione per il test
def test(model, test_loader, criterion, device, label_mapping):
    model.eval()
    running_loss = 0.0
    true_labels = []
    predicted_labels = []

    with torch.no_grad():
        for inputs, labels in tqdm(test_loader, desc='Testing', leave=False):
            inputs, labels = inputs.to(device), labels.to(device)

            outputs = model(inputs)
            loss = criterion(outputs, labels)
            running_loss += loss.item()

            _, preds = torch.max(outputs, 1)
            true_labels.extend(labels.cpu().numpy())
            predicted_labels.extend(preds.cpu().numpy())

    average_loss = running_loss / len(test_loader)
    accuracy, metrics_per_class = calculate_metrics_per_class(true_labels, predicted_labels, label_mapping)

    return average_loss, accuracy, metrics_per_class

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Carica il modello con i pesi migliori
best_model = EmotionCNN()
best_model.load_state_dict(torch.load(r'C:\Users\marco\OneDrive\Documenti\CV_project\ComputerVisionProject\Models\paper1_models\best_model_paper_1_20_epochs_bs_48_30k.pth', map_location=torch.device('cpu')))
best_model.to(device)
criterion = nn.CrossEntropyLoss()
your_label_mapping = {0: 'Angry', 1: 'Disgust', 2: 'Fear', 3: 'Happy', 4: 'Neutral', 5: 'Sad', 6: 'Surprise'}


# Test
test_loss, test_accuracy, test_metrics_per_class = test(best_model, test_loader, criterion, device, your_label_mapping)
test_loss_, test_accuracy_, test_metrics_per_class_ = test(best_model, test_loader_, criterion, device, your_label_mapping)

# Print metrics per class per il test set
print(f'Test Loss: {test_loss:.4f}, Test Accuracy: {test_accuracy:.4f}')
print(f'Test Loss: {test_loss_:.4f}, Test Accuracy: {test_accuracy_:.4f}')

for idx, label in your_label_mapping.items():
    print(f'{label}: Test Precision: {test_metrics_per_class[idx]["precision"]:.4f}, Test Recall: {test_metrics_per_class[idx]["recall"]:.4f}, Test F1: {test_metrics_per_class[idx]["f1"]:.4f}, Test Support: {test_metrics_per_class[idx]["support"]}')


Test Loss: 1.4909, Test Accuracy: 0.6717
Test Loss: 1.7281, Test Accuracy: 0.4297
Angry: Test Precision: 0.6549, Test Recall: 0.5835, Test F1: 0.6171, Test Support: 2000
Disgust: Test Precision: 0.9018, Test Recall: 0.8546, Test F1: 0.8776, Test Support: 1988
Fear: Test Precision: 0.6561, Test Recall: 0.4019, Test F1: 0.4985, Test Support: 2003
Happy: Test Precision: 0.7636, Test Recall: 0.7332, Test F1: 0.7481, Test Support: 2080
Neutral: Test Precision: 0.5245, Test Recall: 0.6624, Test F1: 0.5855, Test Support: 1887
Sad: Test Precision: 0.5437, Test Recall: 0.6002, Test F1: 0.5706, Test Support: 2011
Surprise: Test Precision: 0.6984, Test Recall: 0.8621, Test F1: 0.7717, Test Support: 2031
